In [ ]:
#Returns dataframe with only categorical columns
def check_categorical(dataset):
    cat_columns = dataset.select_dtypes(include=['object','category']).columns
    return dataset[cat_columns]
check_categorical(dataset)

In [ ]:
#Returns dataframe with only numerical columns
def check_numerical(dataset):
    num_columns = dataset.select_dtypes(include=np.number).columns
    return dataset[num_columns]
check_numerical(dataset)

In [ ]:
#Returns columns with number of missing data in each 
def missing_values(dataframe):
    percent_null = (dataframe.isnull().sum()/dataframe.shape[0])*100
    missing_data = pd.Series(percent_null,index=dataframe.columns)
    return missing_data
missing_values(dataset)

In [ ]:
#Returns skew() values for each continous feature:
def check_skewness(dataset):
    numerical_df = check_numerical(dataset)
    for i in numerical_df.columns:
        print(i,numerical_df[i].skew())
check_skewness(dataset)

In [ ]:
#remove skewness, add condition for left skewmess if needed
def remove_skewness(dataset):
    numerical_df = check_numerical(dataset)
    for i in numerical_df.columns:
        if dataset[i].skew()>1:
            dataset[i] = np.log1p(dataset[i])
    return dataset
transformed_dataset = remove_skewness(dataset)
check_skewness(transformed_dataset) 

In [ ]:
#plot distplots for the continous features
def plot_distplots(dataset):
    numerical_df = check_numerical(dataset)
    numerical_columns = numerical_df.columns.tolist()
    for i in range(0,len(numerical_columns),2):
        if len(numerical_columns) > i+1:
            plt.figure(figsize=(10,4))
            plt.subplot(121)
            sns.distplot(dataset[numerical_columns[i]],color='#ffa600')
            plt.subplot(122)            
            sns.distplot(dataset[numerical_columns[i+1]],color='#ffa600')
            plt.tight_layout()
            plt.show()

        else:
            sns.distplot(dataset[numerical_columns[i]],color='#ffa600')
plot_distplots(dataset)

In [ ]:
#bar plots for categorical features
def plot_categorical_columns(dataframe):
    categorical_columns = dataframe.select_dtypes(include=['object']).columns
    
    for i in range(0,len(categorical_columns),2):
            if len(categorical_columns) > i+1:
                
                plt.figure(figsize=(10,4))
                plt.subplot(121)
                dataframe[categorical_columns[i]].value_counts(normalize=True).plot(kind='bar')
                plt.title(categorical_columns[i])
                plt.subplot(122)     
                dataframe[categorical_columns[i+1]].value_counts(normalize=True).plot(kind='bar')
                plt.title(categorical_columns[i+1])
                plt.tight_layout()
                plt.show()

            else:
                dataframe[categorical_columns[i]].value_counts(normalize=True).plot(kind='bar')
                plt.title(categorical_columns[i])
        
        
        
        
plot = plot_categorical_columns(dataset)

In [ ]:
#bivariate countplots for categorical features vs categorical target
def bivariate_analysis_categorical(dataframe,target):
    dataframe = dataframe.drop('Attrition',1)
    categorical_columns = dataframe.select_dtypes(exclude=np.number).columns
    for i in range(0,len(categorical_columns),2):
        
        if len(categorical_columns) > i+1:
            plt.figure(figsize=(15,5))
            plt.subplot(121)
            sns.countplot(x=dataframe[categorical_columns[i]],hue=target,data=dataframe)
            plt.xticks(rotation=90)
            plt.subplot(122)            
            sns.countplot(dataframe[categorical_columns[i+1]],hue=target,data=dataframe)
            plt.xticks(rotation=90)
            plt.tight_layout()
            plt.show()


bivariate_analysis_categorical(dataset,dataset['Attrition'])

In [ ]:
#bivariate plots for continous features vs categorical target
def bivariate_analysis_numerical(dataframe):
    dataset = check_numerical(dataframe)
    numerical_columns = dataset.columns
    for i in numerical_columns:
        plt.figure(figsize=(10,5))
        sns.barplot(x=dataframe['Attrition'],y=dataset[i])
        plt.xticks(rotation=90)
        plt.tight_layout()
        plt.show()


bivariate_analysis_numerical(dataset)

In [ ]:
#returns percentage of classes in the categorical target
def class_imbalance(target):
    class_values = (target.value_counts()/target.value_counts().sum())*100
    return class_values

class_imbalance(dataset['Attrition'])